In [56]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Ign:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy Release
Fetched 257 kB in 1s (189 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to prov

In [57]:
# Import packages
from pyspark.sql import SparkSession
import time
from pyspark import SparkFiles
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [58]:
URL = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [59]:
# 1. Read in the AWS S3 bucket into a DataFrame.
spark.sparkContext.addFile(URL)
file_name = os.path.basename(URL)
file_path = 'file://' + SparkFiles.get(file_name)
print(file_path)
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show()

file:///tmp/spark-9d16e61e-e2f9-410b-a236-02547db0af36/userFiles-8d265502-bfea-4a07-8804-deda8e549f45/home_sales_revised.csv
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|

In [60]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")
spark.sql("SELECT * FROM home_sales LIMIT 5").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [61]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query_4br = """
SELECT YEAR(date) AS year_sold, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year_sold
"""
spark.sql(query_4br).show()

+---------+-------------+
|year_sold|average_price|
+---------+-------------+
|     2022|    296363.88|
|     2019|     300263.7|
|     2020|    298353.78|
|     2021|    301819.44|
+---------+-------------+



In [62]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query_3br_3bath = """
SELECT date_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
"""
spark.sql(query_3br_3bath).show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|     288770.3|
|      2013|    295962.27|
|      2014|    290852.27|
|      2012|    293683.19|
|      2016|    290555.07|
|      2010|    292859.62|
|      2011|    291117.47|
|      2017|    292676.79|
+----------+-------------+



In [63]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query_3br_3bath_2floors = """
SELECT date_built, round(avg(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
"""
spark.sql(query_3br_3bath_2floors).show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|    297609.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2012|    307539.97|
|      2016|     293965.1|
|      2010|    285010.22|
|      2011|    276553.81|
|      2017|    280317.58|
+----------+-------------+



In [64]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query_view = """
SELECT view, round(avg(price), 2) AS average_price
FROM home_sales
GROUP BY view having ROUND(AVG(price),2) >= 350000
"""
spark.sql(query_view).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  81|   1053472.79|
|  76|   1058802.78|
|  91|   1137372.73|
|  93|   1026006.06|
|  52|    733780.26|
|  86|   1070444.25|
|  94|    1033536.2|
|  57|     734340.5|
|  54|    798684.82|
|  96|   1017815.92|
|  92|    970402.55|
|  64|    767036.67|
|  61|    746877.59|
|  88|   1031719.35|
|  72|    780914.67|
|  59|     791453.0|
+----+-------------+
only showing top 20 rows

--- 1.8976449966430664 seconds ---


In [65]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [66]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [67]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query_view = """
SELECT view, round(avg(price), 2)
FROM home_sales
GROUP BY view having ROUND(AVG(price),2) >= 350000
"""
spark.sql(query_view).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  81|          1053472.79|
|  76|          1058802.78|
|  91|          1137372.73|
|  93|          1026006.06|
|  52|           733780.26|
|  86|          1070444.25|
|  94|           1033536.2|
|  57|            734340.5|
|  54|           798684.82|
|  96|          1017815.92|
|  92|           970402.55|
|  64|           767036.67|
|  61|           746877.59|
|  88|          1031719.35|
|  72|           780914.67|
|  59|            791453.0|
+----+--------------------+
only showing top 20 rows

--- 1.2245540618896484 seconds ---


In [68]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

df.write.parquet('home_parquet', mode='overwrite')

In [69]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('home_parquet')

In [70]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('temp_parquet')

In [71]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
query_view = """
SELECT view, round(avg(price), 2) AS temp_parquet
FROM temp_parquet
GROUP BY view having ROUND(AVG(price),2) >= 350000
"""
spark.sql(query_view).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|temp_parquet|
+----+------------+
|  85|  1056336.74|
|  65|   736679.93|
|  53|    755214.8|
|  78|  1080649.37|
|  81|  1053472.79|
|  76|  1058802.78|
|  91|  1137372.73|
|  93|  1026006.06|
|  52|   733780.26|
|  86|  1070444.25|
|  94|   1033536.2|
|  57|    734340.5|
|  54|   798684.82|
|  96|  1017815.92|
|  92|   970402.55|
|  64|   767036.67|
|  61|   746877.59|
|  88|  1031719.35|
|  72|   780914.67|
|  59|    791453.0|
+----+------------+
only showing top 20 rows

--- 0.8105964660644531 seconds ---


In [72]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [73]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
  print('home_sales is cached')
else:
  print('not cached')

not cached
